# Biomass Data Exploration

This notebook explores the biomass and satellite data for understanding spatial patterns and data characteristics.

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import rasterio
from pathlib import Path

# Add src to path
sys.path.append('../src')

from preprocessing.config import SpatialAwarePreprocessingConfig
from utils.data_utils import load_preprocessed_data
from utils.visualization import plot_data_distribution

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

%matplotlib inline

## 1. Load Configuration and Data Paths

In [ ]:
# Load configuration
config = SpatialAwarePreprocessingConfig()

print("Configured raster pairs:")
for i, (sat_path, bio_path, site_name) in enumerate(config.raster_pairs):
    print(f"{i+1}. {site_name}:")
    print(f"   Satellite: {os.path.basename(sat_path)}")
    print(f"   Biomass: {os.path.basename(bio_path)}")
    print(f"   Exists: Sat={os.path.exists(sat_path)}, Bio={os.path.exists(bio_path)}")
    print()

## 2. Examine Raw Raster Data

In [ ]:
# Examine the first raster pair that exists
for sat_path, bio_path, site_name in config.raster_pairs:
    if os.path.exists(sat_path) and os.path.exists(bio_path):
        print(f"Examining {site_name} data...")
        
        # Open satellite data
        with rasterio.open(sat_path) as sat_src:
            print(f"\nSatellite data ({os.path.basename(sat_path)}):")
            print(f"  Shape: {sat_src.shape}")
            print(f"  Bands: {sat_src.count}")
            print(f"  Data type: {sat_src.dtypes[0]}")
            print(f"  CRS: {sat_src.crs}")
            print(f"  Bounds: {sat_src.bounds}")
            
            # Read first band for inspection
            band1 = sat_src.read(1)
            print(f"  Band 1 stats: min={np.nanmin(band1):.2f}, max={np.nanmax(band1):.2f}, mean={np.nanmean(band1):.2f}")
            print(f"  NaN pixels: {np.sum(np.isnan(band1))} ({100*np.sum(np.isnan(band1))/band1.size:.1f}%)")
        
        # Open biomass data
        with rasterio.open(bio_path) as bio_src:
            print(f"\nBiomass data ({os.path.basename(bio_path)}):")
            print(f"  Shape: {bio_src.shape}")
            print(f"  Bands: {bio_src.count}")
            print(f"  Data type: {bio_src.dtypes[0]}")
            print(f"  CRS: {bio_src.crs}")
            print(f"  Bounds: {bio_src.bounds}")
            
            # Read biomass data
            biomass = bio_src.read(1)
            print(f"  Biomass stats: min={np.nanmin(biomass):.2f}, max={np.nanmax(biomass):.2f}, mean={np.nanmean(biomass):.2f}")
            print(f"  NaN pixels: {np.sum(np.isnan(biomass))} ({100*np.sum(np.isnan(biomass))/biomass.size:.1f}%)")
            print(f"  Zero pixels: {np.sum(biomass == 0)} ({100*np.sum(biomass == 0)/biomass.size:.1f}%)")
        
        break
else:
    print("No valid raster pairs found. Please check your data paths.")

## 3. Visualize Raw Data

In [ ]:
# Visualize the first available raster pair
for sat_path, bio_path, site_name in config.raster_pairs:
    if os.path.exists(sat_path) and os.path.exists(bio_path):
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        fig.suptitle(f'Raw Data Visualization - {site_name}', fontsize=16)
        
        # Load data
        with rasterio.open(sat_path) as sat_src:
            sat_data = sat_src.read()
        
        with rasterio.open(bio_path) as bio_src:
            bio_data = bio_src.read(1)
        
        # Plot first few satellite bands
        for i in range(min(3, sat_data.shape[0])):
            im = axes[0, i].imshow(sat_data[i], cmap='viridis')
            axes[0, i].set_title(f'Satellite Band {i+1}')
            axes[0, i].axis('off')
            plt.colorbar(im, ax=axes[0, i], shrink=0.6)
        
        # Plot biomass
        im = axes[1, 0].imshow(bio_data, cmap='YlOrRd')
        axes[1, 0].set_title('Biomass Data')
        axes[1, 0].axis('off')
        plt.colorbar(im, ax=axes[1, 0], shrink=0.6)
        
        # Plot biomass histogram
        valid_biomass = bio_data[~np.isnan(bio_data) & (bio_data > 0)]
        axes[1, 1].hist(valid_biomass, bins=50, alpha=0.7, edgecolor='black')
        axes[1, 1].set_title('Biomass Distribution')
        axes[1, 1].set_xlabel('Biomass')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].grid(True, alpha=0.3)
        
        # Plot log-transformed biomass histogram
        log_biomass = np.log(valid_biomass + 1)
        axes[1, 2].hist(log_biomass, bins=50, alpha=0.7, edgecolor='black', color='orange')
        axes[1, 2].set_title('Log-Transformed Biomass')
        axes[1, 2].set_xlabel('Log(Biomass + 1)')
        axes[1, 2].set_ylabel('Frequency')
        axes[1, 2].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        break

## 4. Load Processed Data (if available)

In [ ]:
# Try to load processed data
try:
    from models.config import HybridCVConfig
    train_config = HybridCVConfig()
    
    data = load_preprocessed_data(train_config)
    
    print("Processed data loaded successfully!")
    print(f"X shape: {data['X'].shape}")
    print(f"y shape: {data['y'].shape}")
    print(f"Number of samples: {len(data['y'])}")
    print(f"Number of sites: {len(np.unique(data['sources']))}")
    
    processed_data_available = True
    
except Exception as e:
    print(f"Processed data not available: {e}")
    print("Run preprocessing first using: python scripts/run_preprocessing.py")
    processed_data_available = False

## 5. Analyze Processed Data

In [ ]:
if processed_data_available:
    # Basic statistics
    X, y = data['X'], data['y']
    sources = data['sources']
    coordinates = data['coordinates']
    
    print("Data Statistics:")
    print(f"Number of bands: {X.shape[1]}")
    print(f"Chip size: {X.shape[2]}x{X.shape[3]}")
    print(f"Biomass range: {np.min(y):.3f} - {np.max(y):.3f}")
    print(f"Biomass mean: {np.mean(y):.3f} ± {np.std(y):.3f}")
    
    # Site breakdown
    unique_sites = np.unique(sources)
    print("\nSite breakdown:")
    for site in unique_sites:
        count = np.sum(sources == site)
        site_biomass = y[sources == site]
        print(f"  Site {site}: {count} samples, biomass: {np.mean(site_biomass):.3f} ± {np.std(site_biomass):.3f}")

## 6. Visualize Processed Data Distribution

In [ ]:
if processed_data_available:
    # Create comprehensive data visualization
    fig, axes = plt.subplots(3, 3, figsize=(18, 15))
    fig.suptitle('Processed Data Analysis', fontsize=16)
    
    # 1. Biomass distribution
    axes[0, 0].hist(y, bins=50, alpha=0.7, edgecolor='black')
    axes[0, 0].set_title('Biomass Distribution (Log Scale)')
    axes[0, 0].set_xlabel('Log(Biomass + 1)')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Biomass by site
    site_data = [y[sources == site] for site in unique_sites]
    axes[0, 1].boxplot(site_data, labels=[f'Site {s}' for s in unique_sites])
    axes[0, 1].set_title('Biomass by Site')
    axes[0, 1].set_ylabel('Log(Biomass + 1)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Sample count by site
    site_counts = [np.sum(sources == site) for site in unique_sites]
    axes[0, 2].bar([f'Site {s}' for s in unique_sites], site_counts)
    axes[0, 2].set_title('Sample Count by Site')
    axes[0, 2].set_ylabel('Count')
    axes[0, 2].tick_params(axis='x', rotation=45)
    axes[0, 2].grid(True, alpha=0.3)
    
    # 4. Spatial distribution
    x_coords = [coord[0] for coord in coordinates]
    y_coords = [coord[1] for coord in coordinates]
    scatter = axes[1, 0].scatter(x_coords, y_coords, c=sources, alpha=0.6, s=10, cmap='tab10')
    axes[1, 0].set_title('Spatial Distribution by Site')
    axes[1, 0].set_xlabel('X Coordinate')
    axes[1, 0].set_ylabel('Y Coordinate')
    plt.colorbar(scatter, ax=axes[1, 0])
    
    # 5. Biomass spatial pattern
    scatter = axes[1, 1].scatter(x_coords, y_coords, c=y, alpha=0.6, s=10, cmap='viridis')
    axes[1, 1].set_title('Spatial Distribution of Biomass')
    axes[1, 1].set_xlabel('X Coordinate')
    axes[1, 1].set_ylabel('Y Coordinate')
    plt.colorbar(scatter, ax=axes[1, 1])
    
    # 6. Band statistics
    band_means = [np.mean(X[:, i, :, :]) for i in range(min(X.shape[1], 10))]
    band_stds = [np.std(X[:, i, :, :]) for i in range(min(X.shape[1], 10))]
    x_bands = range(len(band_means))
    axes[1, 2].errorbar(x_bands, band_means, yerr=band_stds, marker='o')
    axes[1, 2].set_title('Band Statistics (Mean ± Std)')
    axes[1, 2].set_xlabel('Band Index')
    axes[1, 2].set_ylabel('Value')
    axes[1, 2].grid(True, alpha=0.3)
    
    # 7. Sample chip visualization
    sample_idx = np.random.randint(0, len(X))
    sample_chip = X[sample_idx, 0, :, :] if X.shape[1] > 0 else np.zeros((24, 24))
    im = axes[2, 0].imshow(sample_chip, cmap='viridis')
    axes[2, 0].set_title(f'Sample Chip (Band 1)\nBiomass: {y[sample_idx]:.3f}')
    axes[2, 0].axis('off')
    plt.colorbar(im, ax=axes[2, 0], shrink=0.6)
    
    # 8. Correlation between first few bands
    if X.shape[1] >= 3:
        band_corr = np.corrcoef([X[:, i, :, :].flatten() for i in range(min(5, X.shape[1]))])
        im = axes[2, 1].imshow(band_corr, cmap='RdBu_r', vmin=-1, vmax=1)
        axes[2, 1].set_title('Band Correlation Matrix')
        axes[2, 1].set_xticks(range(band_corr.shape[0]))
        axes[2, 1].set_yticks(range(band_corr.shape[0]))
        axes[2, 1].set_xticklabels([f'B{i+1}' for i in range(band_corr.shape[0])])
        axes[2, 1].set_yticklabels([f'B{i+1}' for i in range(band_corr.shape[0])])
        plt.colorbar(im, ax=axes[2, 1], shrink=0.6)
    
    # 9. Data quality assessment
    nan_counts = [np.sum(np.isnan(X[:, i, :, :])) for i in range(X.shape[1])]
    axes[2, 2].bar(range(len(nan_counts)), nan_counts)
    axes[2, 2].set_title('NaN Count by Band')
    axes[2, 2].set_xlabel('Band Index')
    axes[2, 2].set_ylabel('NaN Count')
    axes[2, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 7. Next Steps

Based on this exploration:

1. **If data looks good**: Proceed with training using `02_preprocessing_demo.ipynb`
2. **If issues found**: Adjust preprocessing parameters in configuration files
3. **For training**: Use `python scripts/run_training.py` or `03_model_evaluation.ipynb`

Key things to check:
- All raster files exist and can be opened
- Biomass values are reasonable (not all zeros or NaN)
- Satellite data has reasonable value ranges
- Spatial distribution makes sense
- Sufficient samples per site for cross-validation